Import libraries

In [2]:
import torch
import torch.nn as nn
import tiktoken

Confriguration considered

In [3]:
GPT_CONFIG_124M = {
    "vocab_size": 50527,
    "context_length": 1024,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers" :12,
    "drop_rate": 0.1,
    "qkv_bias": False
}

Multi head attention class

In [4]:
class MultiHeadAttention(nn.Module):

    def __init__(self,d_in,d_out,context_length,dropout=0.5,num_heads=2 ,qkvbias = False):

        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"
        
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads

        self.W_query = nn.Linear(d_in,d_out,qkvbias)
        self.W_key = nn.Linear(d_in,d_out,qkvbias)
        self.W_value = nn.Linear(d_in,d_out,qkvbias)
        self.out_proj = nn.Linear(d_out,d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer("mask", torch.tril(torch.ones(context_length, context_length),diagonal=1))

    def forward(self, x):

        b, num_tokens , d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        keys = keys.view(b,num_tokens,self.num_heads,self.head_dim)
        queries = queries.view(b,num_tokens,self.num_heads,self.head_dim)
        values = values.view(b,num_tokens,self.num_heads,self.head_dim)

        keys = keys.transpose(1,2)  # (b, num_heads, num_tokens, head_dim)
        queries = queries.transpose(1,2)  # (b, num_heads, num_tokens, head_dim)
        values = values.transpose(1,2)  # (b, num_heads, num_tokens, head_dim)

        attn_scores = queries @ keys.transpose(2,3)

        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        attn_scores = attn_scores.masked_fill_(mask_bool,-torch.inf)

        attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5,dim =-1)
        attn_weights = self.dropout(attn_weights)

        context_vector = (attn_weights @ values ).transpose(1,2) #Shape: (b, num_tokens, num_heads, head_dim)

        context_vector = context_vector.contiguous().view(b,num_tokens,self.d_out)
        context_vector = self.out_proj(context_vector)

        return context_vector

Layer normalization class

In [5]:
class LayerNorm(nn.Module):
    def __init__(self,emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim = True)
        variance = x.var(dim = -1, keepdim =True)
        norm_x = (x-mean) / torch.sqrt(variance + self.eps)
        return self.scale * norm_x + self.shift
    

GeLU activation function class

In [6]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5*x*(1+torch.tanh(torch.sqrt(torch.tensor(2.0/torch.pi)) * 
        (x + 0.0044715*torch.pow(x,3))))

Feed forward neural network class

In [7]:
class FeedForward(nn.Module):
    def __init__(self,cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"])
        )

    def forward(self, x):
        return self.layers(x)
    

Implementation of LLM tranformer block

In [21]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.attn = MultiHeadAttention(
            d_in = cfg['emb_dim'],
            d_out = cfg['emb_dim'],
            context_length = cfg['context_length'],
            num_heads = cfg['n_heads'],
            dropout = cfg['drop_rate'],
            qkvbias = cfg['qkv_bias'])

        self.ffn = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg['emb_dim'])
        self.norm2 = LayerNorm(cfg['emb_dim'])
        self.dropout_shortcut = nn.Dropout(cfg['drop_rate'])

    def forward(self,x):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        x = self.attn(x)
        x = self.dropout_shortcut(x)
        x = x + shortcut

        # Shortcut connection for feedforward block
        shortcut =x
        x = self.norm2(x)
        x = self.ffn(x)
        x = self.dropout_shortcut(x)
        x = x + shortcut    

        return x



In [22]:
torch.manual_seed(123)
x = torch.randn(2,4,768)
block = TransformerBlock(GPT_CONFIG_124M)
output = block(x)
print(output.shape)  # Should be (2, 4, 768)

torch.Size([2, 4, 768])
